Сбор данных

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
import time 
import pandas as pd
 
def scrap_page(df_entire):
    '''Функция парсинга одной страницы'''
    
    poems = driver.find_elements(By.CLASS_NAME, 'poems')
    tatar = poems[0]
    russian = poems[1]

    tatar_poems = [x.text for x in tatar.find_elements(By.CLASS_NAME, 'poem-content')]
    russian_poems = [x.text for x in russian.find_elements(By.CLASS_NAME, 'poem-content')]
    if len(tatar_poems) == len(russian_poems): 
        df_new = pd.DataFrame({'tatar': tatar_poems, 'russian': russian_poems})
        df_entire = pd.concat([df_entire, df_new], ignore_index=True)
        
    return df_entire

def go_to_next_page():
    '''Функция перехода на следующую страницу'''
    
    try:
        next_button = driver.find_element(By.CLASS_NAME, 'next-chapter')
        next_button.click()
        return True
    
    except NoSuchElementException as e:
        return False
    
def get_urls():
    '''Функция сбора адресов первых страниц каждой книги'''
    
    base_url = 'https://online-bibliya.ru/book/tatarskii-tatarcha-34/ot-matfeya-1458/1/compare/sovremennyi-9'
    driver.get(base_url)
    base_urls =[a.get_attribute('href') for a in driver.find_elements(By.XPATH, '//div[@itemtype="http://schema.org/Book"]/a')]

    return base_urls


base_url = 'https://online-bibliya.ru/book/tatarskii-tatarcha-34/ot-matfeya-1458/1/compare/sovremennyi-9'

driver = webdriver.Chrome()

driver.get(base_url)

time.sleep(1)

book_urls = get_urls()
df = pd.DataFrame(columns=['tatar', 'russian'])

for url in book_urls:
    driver.get(url)
    dummy_url = url.replace("/1/compare", "/{}/compare")
    
    total_pages = len(driver.find_element(By.CLASS_NAME, 'chapters').find_elements(By.TAG_NAME, 'a'))
    for i in range(total_pages):
        cur_url = dummy_url.format(i+1)
        driver.get(cur_url)
        time.sleep(1)
        df = scrap_page(df)

driver.quit()

df.to_csv('bibble.csv', index=False)

In [2]:
df.head()

,tatar,russian
0,Ибраһим һәм Давыт токымыннан булган Гайсә Мәси...,"Вот родословная Иисуса Христа, Происходящего и..."
1,"Ибраһим Исхакның, Исхак Ягъкубның, Ягъкуб Яһүд...",Авраам был отцом Исаака. Исаак был отцом Иаков...
2,"Яһүд Тамардан туган Перес һәм Зерахның әтисе, ...","Иуда был отцом Фареса и Зары, чьей матерью был..."
3,"Рам Амминадабның, Амминадаб Нахшунның, Нахшун ...",Арам был отцом Аминадава. Аминадав был отцом Н...
4,"Салмун Рәхәбтән туган Богазның, Богаз Руттан т...","Салмон был отцом Вооза, чьей матерью была Раха..."


In [3]:
df.shape

(7780, 2)

Парсинг текста занял 7 мин 13.2 с 

 7780 строк

In [4]:
from datasets import load_dataset
data = load_dataset('AigizK/tatar-russian-parallel-corpora')
df2 = data['train'].to_pandas()
df2.rename(columns={'tat': 'tatar', 'rus': 'russian'}, inplace=True)

In [5]:
df2.head()

,tatar,russian
0,"""Рэп-баттл ярдәмендә фигыльләрне өйрәнәбез"".",«Учим глаголы при помощи рэп-баттла».
1,"""Россия өчен укытучы"" проектында катнашучылар ...",Участники проекта «Учитель для России» — о сел...
2,"2015 елда берничә төбәктә ""Россия өчен укытучы...",В 2015 году в нескольких регионах стартовала п...
3,Нәкъ шундый ук проектлар күп кенә илләрдә — Һи...,Точно такие же проекты существуют во многих ст...
4,Болар барысы да шул исәптән мәктәпкә мөнәсәбәт...,"Все это придумано в том числе и для того, чтоб..."


In [6]:
df2.shape

(161831, 2)

В данном датасете 161831 строка

In [7]:
df_all = pd.concat([df, df2], ignore_index=True)
df_all.rename(columns={'tatar': 'Tatar', 'russian': 'Russian'}, inplace=True)

In [8]:
df_all.head()

,Tatar,Russian
0,Ибраһим һәм Давыт токымыннан булган Гайсә Мәси...,"Вот родословная Иисуса Христа, Происходящего и..."
1,"Ибраһим Исхакның, Исхак Ягъкубның, Ягъкуб Яһүд...",Авраам был отцом Исаака. Исаак был отцом Иаков...
2,"Яһүд Тамардан туган Перес һәм Зерахның әтисе, ...","Иуда был отцом Фареса и Зары, чьей матерью был..."
3,"Рам Амминадабның, Амминадаб Нахшунның, Нахшун ...",Арам был отцом Аминадава. Аминадав был отцом Н...
4,"Салмун Рәхәбтән туган Богазның, Богаз Руттан т...","Салмон был отцом Вооза, чьей матерью была Раха..."


In [9]:
df_all.shape

(169611, 2)

После объединения получен датасет из 169611 строки

In [11]:
df_all.to_csv('all_data.csv', index=False)